<center>

**Vježba 3 - Linearna regresija i MLflow**

</center>

U ovoj vježbi želimo pokazati dodatne funkcionalnosti *MLflow* servera.

Prvo učitavamo potrebne python module.

In [ ]:
%%bash
pip install matplotlib

In [ ]:
from sklearn import datasets
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import numpy as np
import pandas as pd

import mlflow
import mlflow.sklearn
from mlflow.models.signature import infer_signature
from mlflow.client import MlflowClient
from mlflow.store.artifact.runs_artifact_repo import RunsArtifactRepository

Učitavamo *diabetes* skup podataka iz *scikit-learn* modula i ujedno ga pretvaramo u pandas DataFrame (nema automatike).  
Zatim definiramo funkciju za custom metriku modela.

In [ ]:
def loadDiabetesDataset():
    diabetes = datasets.load_diabetes()
    X = diabetes.data
    y = diabetes.target

    Y = np.array([y]).transpose()
    d = np.concatenate((X, Y), axis=1)
    cols = diabetes.feature_names + ["progression"]
    data = pd.DataFrame(d, columns=cols)
    return data

def evalMetrics(actual, pred):
    rmse = np.sqrt(mean_squared_error(actual, pred))
    mae = mean_absolute_error(actual, pred)
    r2 = r2_score(actual, pred)
    return rmse, mae, r2

Postavimo *MLflow* server URL. Zatim potražimo naš '%Diabetes%' eksperiment.  
U slučaju da eksperiment nije pronađen, stvorimo ga.

In [ ]:
mlflow.set_tracking_uri("http://localhost:5000")
exps = mlflow.search_experiments(filter_string="name LIKE '%Diabetes%'")
if len(exps)==0:
    exp_id = mlflow.create_experiment("Linear regression - Diabetes dataset")
else:
    exp_id = exps[0].experiment_id

Učitamo skup podataka i podijelimo ga na podskup za učenje i testiranje.

In [ ]:
data = loadDiabetesDataset()
train, test = train_test_split(data)

Prvo naučimo linearni regresor na podskupu za učenje.  
Zatim detektiramo sučelje regresora na temelju podskupa za učenje.  

Svaki poziv *MLflow-a* funkcionira kao posebno *pokretanje - run*. U našem slučaju smo pokretanju dali posebni naziv s kojim uočavamo da se radi o učenju modela.  
U našem pokretanju zapišemo model koji smo prethodno naučili.  
Posebno definiramo meta-podatke o modelu koji smo upisali.

In [ ]:
train_x = train.drop(["progression"], axis=1)
train_y = train[["progression"]]

lr = LinearRegression()
lr.fit(train_x, train_y)

schema = infer_signature(train_x, train_y)
with mlflow.start_run(run_name="Training diabetes linear regression model",
                        experiment_id=exp_id) as run:
    mlflow.sklearn.log_model(lr, artifact_path="sklearn-model", signature=schema)
        
name="Diabetes linear regression model"

client = MlflowClient()
rms = client.search_registered_models("name='"+name+"'")
if len(rms)==0: client.create_registered_model(name)

model_uri = "runs:/{}/sklearn-model".format(run.info.run_id)
model_src = RunsArtifactRepository.get_underlying_uri(model_uri)
mv = client.create_model_version(name, model_src, run.info.run_id, 
    description="Linear regression model for diabetes dataset")

mlflow.end_run()


Zatim taj naučeni model automatski evaluiramo na *MLflow* serveru.  
Tu evaluaciju definiramo kao posebno pokretanje specifičnog naziva.  

In [ ]:
client = MlflowClient()
models = client.get_latest_versions(name, stages=["None"])

with mlflow.start_run(run_name="Testing diabetes linear regression model, version:"+models[0].version,
                        experiment_id=exp_id) as run:
    mlflow.evaluate(models[0].source,test,targets="progression",model_type="regressor",
        evaluators=["default"])

mlflow.end_run()

Možemo napraviti i ručnu evaluaciju koju upišemo kao posebno pokretanje.  
Sva pokretanja sada možemo provjeriti na korisničkom sučelju *MLflow* servera.

In [ ]:
client = MlflowClient()
models = client.get_latest_versions(name, stages=["None"])
test_x = test.drop(["progression"], axis=1)
test_y = test[["progression"]]

with mlflow.start_run(run_name="Manual test of diabetes linear regression model, version:"+models[0].version,
                        experiment_id=exp_id) as run:
    lr = mlflow.sklearn.load_model(models[0].source)
    predict = lr.predict(test_x)
    (rmse, mae, r2) = evalMetrics(test_y, predict)
    mlflow.log_metric("RMSE", rmse)
    mlflow.log_metric("MAE", mae)
    mlflow.log_metric("R2", r2)
    
mlflow.end_run()